# 한글 자연어 전처리 실습 문제
> 작성자      
```
* 김성현 (bananaband657@gmail.com)  
김바다 (qkek983@gmail.com)
박상희 (parksanghee0103@gmail.com)  
이정우 (jungwoo.l2.rs@gmail.com)
```

[CC BY-NC-ND](https://creativecommons.org/licenses/by-nc-nd/2.0/kr/)

이번 실습에서는 실제 현업에서 필요로 하는 다양한 전처리를 실습해보도록 하겠습니다! :-)   
중간 중간에 있는 'myFunction' 부분을 채워서 제출해주세요!   
예제 코드는 다음날인 4월 14일에 공개됩니다 🤗   


### 음성처리 관련 문제 (3)
1. 알파벳 및 특수문자를 한글 발음으로 변환하기
2. 대화속에서 등장한 금액과 계좌번호 추출하기
3. 대화속에서 등장한 핸드폰 번호 추출하기

### 개인정보 관련 문제 (2)
4. 개인정보 마스킹하기
5. 욕설 삐처리하기

### 그 외 (2)
6. 특정 영상의 유투브 댓글 크롤링하기
7. Corpus EDA

### 1. 자연어를 한글 발음으로 변환하기

한국어는 특히 자연어를 발음으로 표현할 때, 구개음화 등과 같은 다양한 특징을 가집니다 😂   
음성 합성을 위한 텍스트를 생성할 때, '소리나는 대로' 표현하기 위해 일반적인 자연어 텍스트를 발음 형태로 변형해주기도 합니다 🤗   




In [51]:
!pip install g2pk

In [52]:
from g2pk import G2p
g2p = G2p()

In [53]:
text = "Lost arc는 smile gate가 개발한 action game이다.\n\
2014년 11월 12일 최초 공개했으며 2018년 11월 7일부터 오픈 베타 테스트를 진행하다 2019년 12월 4일 정식 오픈했다."

In [54]:
print(g2p(text))

로스트 아크는 스마일 게이트가 개발하 낵쎤 게이미다.
이영일사녀 니리뤄 리리일 최초 공개해쓰며 이영일팔려 니리뤌 치릴부터 오픈 베타 테스트를 진행하다 이영일구녀 니리월 사일 정시 고픈핻따.


### 2. 대화속에서 등장한 금액과 계좌번호 추출하기

- 고객과 상담사간의 통화 내용을 음성인식한 데이터입니다.   
- 해당 데이터에서 금액과 계좌번호를 추출하는 시나리오입니다.   
- A은행의 계좌번호는 "-"를 포함해서 12자리입니다.

- 예시) xxx-xxx-xxxx

In [55]:
texts = ["여보세요. 거기 A은행이죠? 저의 계좌에 문제가 있어서 연락드렸습니다. 어플로 돈을 송금해야 하는데 오류가 뜨네요.",
         "네! 고객님 안녕하세요. A은행 상담사 홍길동입니다. 계좌번호에 문제가 생기셔서 연락주셨다구요?",
         "네 맞아요",
         "현재 어플 전산망의 문제가 생겨 복구중에 있습니다. 직접 송금 서비스를 도와드리겠습니다. 인출하실 계좌번호를 알려주실 수 있을까요?",
         "일이삼다시사오육다시칠팔구공이요",
         "고객님이 송금하고 싶으신 금액은 얼마입니까?",
         "팔억구천팔백칠십오만육백오십칠원이에요",
         "고객님이 송금하시고자 하는 계좌번호를 알려주실 수 있을까요?",
         "영구팔다시칠육오다시사삼이일이요",
         "네! 고객님 이용해 불편을 드려 대단히 죄송합니다. 송금 정상처리되었습니다.",
         "네 감사합니다."]

In [184]:
text2num = {"일": "1", "이": "2", "삼": "3", "사": "4", "오": "5",
            "육": "6", "칠": "7", "팔": "8", "구": "9", "공": "0",
            "영": "0"}

def text_to_num(text):
    num = []
    for t in text:
      num.append(text2num[t])

    return "".join(num)


def find_account(text):
    is_accout_compile = re.compile("(...)다시(...)다시(....)")
    account = []

    for i in range(1, 3):
        account.append(text_to_num(is_accout_compile.search(text).group(i)))
        account.append("-")
    else:
        account.append(text_to_num(is_accout_compile.search(text).group(3)))
    
    account = "".join(account)

    return account


def text_to_digit(text):
    is_money_extraction_sentence_compile = re.compile("([가-힣]+)원")

    text_money = is_money_extraction_sentence_compile.search(text).group(1)
    unit1 = {"억": 1e8, "만": 1e4}
    unit2 = {"천": 1e3, "백": 1e2, "십": 1e1}
    total_money = 0
    temp_money = 0
    temp_num = 0
    
    for t in text_money:
        if t in text2num:
            temp_num = int(text2num[t])
        elif t in unit2:
            temp_money += temp_num * unit2[t]
        elif t in unit1:
            temp_money += temp_num
            total_money += temp_money * unit1[t]
            temp_money = 0
    else:
        total_money += temp_money + temp_num

    result = ""
    cnt = 0
    for i in str(int(total_money))[::-1]:
        if cnt == 3:
            result += ","
            cnt = 0
        result += i
        cnt += 1
        
    return result[::-1] + "원"


def myFunction(texts):
    is_from_account_sentence = False
    from_account_compile = re.compile("인출|계좌번호")

    is_to_account_sentence = False
    is_to_account_compile = re.compile("송금|금액")

    is_money_extraction_sentence = False
    is_money_extraction_compile = re.compile("송금|계좌번호")

    for text in texts:
        if is_from_account_sentence:
            from_account = find_account(text)
            is_from_account_sentence = False
            continue

        if is_to_account_sentence:
            to_account = find_account(text)
            is_to_account_sentence = False
            continue

        if is_money_extraction_sentence:
            money_extraction = text_to_digit(text)
            # print(is_money_extraction_sentence_compile.search(text).group(1))
            is_money_extraction_sentence = False
            continue

        if set(from_account_compile.findall(text)) == {"인출", "계좌번호"}:
            is_from_account_sentence = True
            continue

        if set(is_to_account_compile.findall(text)) == {"송금", "금액"}:
            is_money_extraction_sentence = True
            continue
        
        if set(is_money_extraction_compile.findall(text)) == {"송금", "계좌번호"}:
            is_to_account_sentence = True
            continue
    
    # 최종 원하는 결과들을 딕셔너리 형태로 리턴
    return {"from_Account" : from_account, "to_Account" : to_account, "money" : money_extraction}

In [185]:
myFunction(texts)

{'from_Account': '123-456-7890',
 'money': '898,750,657원',
 'to_Account': '098-765-4321'}

### 3. 대화속에서 등장한 핸드폰 번호 추출하기

- 고객과 상담사간의 통화 내용을 음성인식한 데이터입니다.   
- 대화 속에 등장한 핸드폰 번호를 추출해보도록 하겠습니다.   

In [186]:
texts = ["XXX 쇼핑몰 고객센터입니다.",
         "배송 관련 문의 사항은 공일공육사칠구팔구삼삼입니다.",
         "환불 및 교환 관련 문의사항은 공삼일구육칠이육삼삼입니다.",
         "제품 풀질 문의사항은 공일일일일이삼일이삼사입니다.", 
         "기타 문의사항은 공삼일팔구구구일이삼사입니다.",
         "상담원 연결은 공이칠칠칠팔이팔이입니다."]

In [187]:
texts

['XXX 쇼핑몰 고객센터입니다.',
 '배송 관련 문의 사항은 공일공육사칠구팔구삼삼입니다.',
 '환불 및 교환 관련 문의사항은 공삼일구육칠이육삼삼입니다.',
 '제품 풀질 문의사항은 공일일일일이삼일이삼사입니다.',
 '기타 문의사항은 공삼일팔구구구일이삼사입니다.',
 '상담원 연결은 공이칠칠칠팔이팔이입니다.']

In [213]:
text2num = {"일": "1", "이": "2", "삼": "3", "사": "4", "오": "5",
            "육": "6", "칠": "7", "팔": "8", "구": "9", "공": "0",
            "영": "0"}

def text_to_num(text):
    num = []
    for t in text:
      num.append(text2num[t])

    num_length = len(num)

    if num_length == 11:
        return "".join(num[:3]) + "-" + "".join(num[3:7]) + "-" + "".join(num[7:])
    elif num_length == 10:
        return "".join(num[:3]) + "-" + "".join(num[3:6]) + "-" + "".join(num[6:])
    elif num_length == 9:
        return "".join(num[:2]) + "-" + "".join(num[2:5]) + "-" + "".join(num[5:])

def myFunction(texts):
    is_phone_number_compile = re.compile("(공[가-힣]+)입니다.")
    phones = []

    for text in texts:
        phone_number = is_phone_number_compile.search(text)
        if phone_number:
            phones.append(text_to_num(phone_number.group(1)))

    # 저장된 전화번호 리스트를 리턴
    return phones

In [214]:
myFunction(texts)

['010-6479-8933',
 '031-967-2633',
 '011-1123-1234',
 '031-8999-1234',
 '02-777-8282']

### 4. 개인정보 마스킹하기

- 개인정보 마스킹의 중요성은 아무리 강조해도 지나치지 않습니다.
- 아래 텍스트들에 대해 개인 정보를 마스킹해보겠습니다.

In [215]:
texts = ["진 김석진 1992-12-04 Seokjin_INTP@gmail.com",
         "슈가 민윤기 1993-03-09 Yunki_INTP@daum.com",
         "제이홉 정호석 1994-02-18 Hoseok_ESFJ@naver.com",
         "RM 김남준 1994-09-12 Namjun_ENFP@naver.com",
         "지민 박지민 1995-10-13 Jimin_ENJF@daum.com",
         "뷔 김태형 1995-12-30 Taehyung_ENFP@hanmail.net",
         "정국 전정국 1997-09-01 Jungkook_ISFP-A@kakak.com"]

In [216]:
texts

['진 김석진 1992-12-04 Seokjin_INTP@gmail.com',
 '슈가 민윤기 1993-03-09 Yunki_INTP@daum.com',
 '제이홉 정호석 1994-02-18 Hoseok_ESFJ@naver.com',
 'RM 김남준 1994-09-12 Namjun_ENFP@naver.com',
 '지민 박지민 1995-10-13 Jimin_ENJF@daum.com',
 '뷔 김태형 1995-12-30 Taehyung_ENFP@hanmail.net',
 '정국 전정국 1997-09-01 Jungkook_ISFP-A@kakak.com']

In [236]:
def myFunction(texts):
    new_texts = []
    for text in texts:
        name1, name2, birth, email = text.split()
        name2 = name2[0] + "*" * len(name2[1:])
        email_to_anonymization, email = email.split("@")
        email = email_to_anonymization[0] + "*" * len(email_to_anonymization[1:]) + "@" + email
        new_texts.append(" ".join([name1, name2, birth, email]))
            
    return new_texts

In [237]:
myFunction(texts)

['진 김** 1992-12-04 S***********@gmail.com',
 '슈가 민** 1993-03-09 Y*********@daum.com',
 '제이홉 정** 1994-02-18 H**********@naver.com',
 'RM 김** 1994-09-12 N**********@naver.com',
 '지민 박** 1995-10-13 J*********@daum.com',
 '뷔 김** 1995-12-30 T************@hanmail.net',
 '정국 전** 1997-09-01 J**************@kakak.com']

### 5.  욕설 삐처리 하기

In [239]:
!git clone https://github.com/2runo/Curse-detection-data.git

Cloning into 'Curse-detection-data'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 16 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (16/16), done.


In [240]:
import random
import pandas as pd
from tqdm import tqdm

data = pd.DataFrame()

f = open("./Curse-detection-data/dataset.txt", 'r')
lines = f.readlines()
for line in tqdm(lines):
    docs = line.replace("\n", "").split("|")
    data = data.append({"text":docs[0], "label":docs[1]}, ignore_index=True)
f.close()

data = data[data["label"] == "1"].reset_index(drop=True)
random_index = random.sample(range(len(data)), 10)
texts = [data.loc[row]["text"] for row in random_index]

100%|██████████| 5825/5825 [00:15<00:00, 373.82it/s]


In [241]:
texts

['운지햐라',
 '그러니깐 딜도로 쑤셔박아야한다는거지?',
 '야이 빙신아 천조국은 석유도 나니까 땅이 기름지지 ㅉㅉ',
 '니도 느검마 눈엔 아기천사였어',
 '하...짱깨야 보고있나??',
 'ㅇㅎ 과거도 현재도 민족의식 애국심 존나 안드는건 마찬가진가지네',
 '그럼 땅에 뭍으라고??? 뭐 어떻게 하라고저런덴 쓰레기 수거를 안해 병신아 ',
 '넌 시발 뭐하는 새끼인데 여기서 이지랄이냐? 기술이나 배워라.',
 '뭘 끼워맞쳐 ㅉ   티모테오   내가 처죽여냐 ?? 지가 처맞아죽었고 그게 기록상 팩트쟌아 그런데 뭘끼어맞쳐다는건지  시비걸지마라http://maria.catholic.or.kr/sa_ho/list/list.asp?menugubun=saint&today=&today_tmp=&ctxtCommand=&ctxtLogOn=&ctxtSexcode=&ctxtChukday=&ctxtGaladay=&Orggubun=101&ctxtHigh=&ctxtLow=&ctxtChecked=Checked&oldrow=&curpage=1&ctxtOrder=name1%2Cgaladaym%2Cgaladayd&ctxtOrderType=&ctxtSaintId=&ctxtSCode=&ctxtSearchNm=%EB%94%94%EB%AA%A8%ED%85%8C%EC%98%A4&ctxtChukmm=&ctxtChukdd=&ctxtPosition=&ctxtCity=&PSIZE=20',
 '시발년이네남친 잇냐?']

In [242]:
# 욕설 사전
!git clone https://github.com/LDNOOBW/List-of-Dirty-Naughty-Obscene-and-Otherwise-Bad-Words
!git clone https://github.com/doublems/korean-bad-words
!git clone https://github.com/organization/Gentleman/

Cloning into 'List-of-Dirty-Naughty-Obscene-and-Otherwise-Bad-Words'...
remote: Enumerating objects: 480, done.
remote: Total 480 (delta 0), reused 0 (delta 0), pack-reused 480
Receiving objects: 100% (480/480), 110.14 KiB | 939.00 KiB/s, done.
Resolving deltas: 100% (249/249), done.
Cloning into 'korean-bad-words'...
remote: Enumerating objects: 56, done.
remote: Total 56 (delta 0), reused 0 (delta 0), pack-reused 56
Unpacking objects: 100% (56/56), done.
Cloning into 'Gentleman'...
remote: Enumerating objects: 55, done.
remote: Total 55 (delta 0), reused 0 (delta 0), pack-reused 55
Unpacking objects: 100% (55/55), done.


In [243]:
import json

# 비속어 사전 01
slang_list_01 = []
f = open("./List-of-Dirty-Naughty-Obscene-and-Otherwise-Bad-Words/ko", 'r')
lines = f.readlines()
for line in lines:
    line = line.replace("\n", "")
    slang_list_01.append(line)
f.close()

# 비속어 사전 02
slang_list_02 = []
f = open("./korean-bad-words/korean-bad-words.md", 'r')
lines = f.readlines()
for line in lines:
    line = line.replace("\n", "")
    slang_list_02.append(line)
f.close()

# 비속어 사전 03
with open('./Gentleman/resources/badwords.json') as json_file:
    json_data = json.load(json_file)
slang_list_03 = json_data["badwords"]

# 비속어 사전 통합
slang_list = slang_list_01 + slang_list_02 + slang_list_03
slang_list = list(set(slang_list))

In [261]:
def myFunction(texts):
    new_texts = []

    for text in texts:
        for slang in slang_list:
            text = text.replace(slang, "**")
        new_texts.append(text)
            
    return new_texts

In [262]:
myFunction(texts)

['운지햐라',
 '그러니깐 **로 쑤셔박아야한다는거지?',
 '야이 **아 천조국은 석유도 나니까 땅이 기름지지 ㅉㅉ',
 '니도 ** 눈엔 아기천사였어',
 '하...**야 보고있나??',
 'ㅇㅎ 과거도 현재도 민족의식 애국심 ** 안드는건 마찬가진가지네',
 '그럼 땅에 뭍으라고??? 뭐 어떻게 하라고저런덴 ** 수거를 안해 **아 ',
 '넌 ** 뭐하는 **인데 여기서 이**냐? 기술이나 배워라.',
 '뭘 끼워맞쳐 ㅉ   티모테오   내가 처죽여냐 ?? 지가 처맞아죽었고 그게 기록상 팩트쟌아 그런데 뭘끼어맞쳐다는건지  시비걸지마라http://maria.catholic.or.kr/sa_ho/list/list.asp?menugubun=saint&today=&today_tmp=&ctxtCommand=&ctxtLogOn=&ctxtSexcode=&ctxtChukday=&ctxtGaladay=&Orggubun=101&ctxtHigh=&ctxtLow=&ctxtChecked=Checked&oldrow=&curpage=1&ctxtOrder=name1%2Cgaladaym%2Cgaladayd&ctxtOrderType=&ctxtSaintId=&ctxtSCode=&ctxtSearchNm=%EB%94%94%EB%AA%A8%ED%85%8C%EC%98%A4&ctxtChukmm=&ctxtChukdd=&ctxtPosition=&ctxtCity=&PSIZE=20',
 '시**이네남친 잇냐?']

### 6. 특정 영상의 유투브 댓글 크롤링하기

1. 셀레니움을 위해서 크롬 드라이버 설치하기

In [270]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!apt-get install xvfb
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
!pip install pyvirtualdisplay

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic

In [271]:
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from pyvirtualdisplay import Display

In [272]:
# display = Display(visible=0, size = (1024,768))
# display.start()
# driver = webdriver.Chrome("/home/roa/project/chromedriver")

display = Display(visible=0, size = (1024,768))
display.start()
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
# driver = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: use options instead of chrome_options
  if sys.path[0] == '':


In [273]:
url = "https://www.youtube.com/watch?v=BS6O0zOGX4E&list=PLlMkM4tgfjnLSOjrEJN31gZATbcj_MpUm"

In [274]:
def myFunction (url, n) : 
    
    # 크롬 드라이버
    driver.get(url)
    
    # 페이지의 가장 아래부분의 창 높이를 저장
    last_page_height = driver.execute_script("return document.documentElement.scrollHeight") 
    
    # 창이 제일 아래에 도달할 때 까지 반복
    while True: 
        
        # 스크롤
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);") 
        
        # 1초 대기
        time.sleep(1.0) 
        
        # 헌재의 스크롤 높이
        new_page_height = driver.execute_script("return document.documentElement.scrollHeight") 
        
        # 현재 높이가 제일 아래이면 반복문 탈출
        if new_page_height == last_page_height: 
            break 
        
        # 현재 페이지 갱신
        last_page_height = new_page_height
    
    # 페이지 소스 가져오기
    html_source = driver.page_source
    
    # lxml을 parsing
    soup = BeautifulSoup(html_source, 'lxml')
        
    # 댓글부분 선택
    comments = soup.select('yt-formatted-string#content-text')
    
    # 텍스트만 정제해서 담을 리스트
    comments_preprossed = []
    
    # 1 ~ n개 까지만 가져옴
    for comment in comments[:n] : 
        
        # 가져온 댓글 내용을 저장
        comments_preprossed.append(comment.text)

    # 저장된 댓글 리턴
    return comments_preprossed

In [275]:
myFunction(url=url, n=10)

['질문이나 강의에 대한 건의등이 있으시면 편안하게 올려 주세요. 다음 강의등에 반영하겠습니다.',
 '눈물 날 정도로 감사합니다',
 '소고기 국수 먹으러 다니시던 분 아니신가?',
 '머신러닝 넘나 재밌는 것..',
 '정주행하겠습니다!!!',
 '감사합니다 센세 흑흑',
 '정주행 시작!',
 '사랑합니다 교수님 ^^',
 '머리속에 쏙쏙 들어오는 명강의!',
 '감사합니다 열심히 듣겠습니다.']

### 7. Corpus EDA

- 사전 언어 모델을 학습하기전에, 학습시킬 Corpus에 대해 이해하는 것은 필수입니다.

- 주어진 Corpus의 문장의 통계값

- 가장 많이 나온 단어

In [276]:
texts = myFunction(url, n=100)

In [277]:
import numpy as np
from collections import Counter

In [278]:
def myFunction (texts) : 
    
    texts_len = []
    
    word_list = []
    
    # 문장의 길이를 저장 및 띄어쓰기 단위로 토큰화
    for text in texts : 
        texts_len.append(len(text))
      
        # 띄어쓰기 토큰
        for word in text.split() : 
            word_list.append(word)
    
    # 문장에 포함된 단어들 카운트
    counter = Counter(word_list)
    
    # 가장 많이 나온 단어 10개
    word_list = counter.most_common(n=10)

    # 빈도는 지우고 단어만, 순서대로 저장
    word_list = [word[0] for word in word_list]
     
    return {"texts" : len(texts),
            "maximum" : np.max(texts_len), "minumum" : np.min(texts_len),
            "mean" : np.mean(texts_len), "median" : np.median(texts_len), "std" : np.std(texts_len),
            "1Q" : np.quantile(texts_len, 0.25), "3Q" : np.quantile(texts_len, 0.75),
            "TOP1_word" : word_list[0], "TOP10_word":word_list[:10]}

In [279]:
myFunction(texts)

{'1Q': 12.0,
 '3Q': 39.5,
 'TOP10_word': ['좋은',
  '감사합니다',
  '감사합니다.',
  '강의',
  '정말',
  '열심히',
  '감사합니다!',
  '강의를',
  '이렇게',
  '교수님'],
 'TOP1_word': '좋은',
 'maximum': 157,
 'mean': 30.98,
 'median': 20.0,
 'minumum': 2,
 'std': 29.044097507066734,
 'texts': 100}